Create a ad hoc network having 10 nodes and a total number of 32 connections.
At each node, generate the data according to the following model:

$$ y_k(n) = \boldsymbol{\theta}_0^T \mathbf{x}_k(n) + \eta_k(n), 1 \leq k \leq 10, $$

where
* $ \boldsymbol{\theta}_0 \in \mathbb{R}^{60} $ is a constant vector, generated via $ N(0,1) $.
* all input vectors $ \mathbf{x}_k(n) \in \mathbb{R}^{60} $ are i.i.d generated via $ N(0,1) $.
* noise samples $ \eta_k(n) \in \mathbb{R}^1 $ are independently generated from zero mean Gaussians with variances corresponding to different signal-to-noise level, varying from 20-25 dBs in each node.

For the unknown vector estimation employ the following algorithms:
* combine-then-adapt diffusion APSM. Parameters: $ \mu_n = 0.5 \times M_n $, $ \epsilon_k = \sqrt 2 \sigma_k $, $ q = 20 $.
* adapt-then-combine LMS. Parameter: step size = 0.03.
* combine-then-adapt LMS. Parameter: step size = 0.03.
* noncooperative LMS. Parameters: choose $ a_{mk} $ according to Metropolis rule:

$$ a_{mk} = \left\{
\begin{aligned}
&\frac{1}{max(n_k,n_m)}, &k \neq m,\ k,\ m\ are\ neighbors, \\
&0, &m \neq k, \\
&1-\sum_{i \neq k} a_{ik}, &m = k.
\end{aligned}
\right.
$$

Run 100 independent experiments and plot the average MSD per iteration in dBs:

$$ MSD(n) = 10 \log_{10} \left( \frac{1}{K} \sum_{k=1}^{K} \Vert \boldsymbol{\theta}_{k}(n) - \boldsymbol{\theta}_0 \Vert^2 \right). $$

* class networkController: generate $ \boldsymbol{\theta}_0 $ and send it to nodes, signal nodes to begin training, record training history, compute MSD.
* class node: generate input vector and noise, run algorithms, communicate with adjacent nodes.

In [1]:
# Import packages
import numpy as np
import scipy as sp
import matplotlib as plt

class networkController: generate $ \boldsymbol{\theta}_0 \in \mathbb{R}^{60} $ via $ N(0,1) $, compute true value according to sample, signal nodes to begin training, record training history, compute MSD.

In [24]:
# networkController
class networkController:
    
    def __init__(self):
        self.true_theta = np.zeros(60) # initialise theta0 in R^60
        self.energy = 0 # initialise energy level of signal
        self.nodes = [networkNode(i,self) for i in range(10)] # initialise nodelist
        self.count_nodes = len(self.nodes)
        
        # initialise training history
        self. caASPM_history = np.array(0)
        self.acLMS_history = np.array(0)
        self.caLMS_history = np.array(0)
        self.ncLMS_history = np.array(0)
    
    # generate theta0 via standard normal distribution
    def genTheta(self):
        self.true_theta = np.random.randn(60)
        self.energy = np.linalg.norm(self.true_theta)**2
        return
    
    # train models for iters iterations using selected algorithm
    def startTrain(self, iters: int, algorithm: str):
        for node in self.nodes:
            node.adjacent_nodes = list(node.adjacent_nodes)
            node.updateArgs()
            node.noise = np.sqrt(self.energy*np.power(10,-node.snr/10))
            
        # combine-then-adapt ASPM
        if algorithm == "caASPM":
            return
        
        # adapt-then-combine LMS
        elif algorithm == "acLMS":
            self.acLMS(iters)
        
        # combine-then-adapt LMS
        elif algorithm == "caLMS":
            return
        
        # noncooperative LMS
        elif algorithm == "ncLMS":
            return
        
        for node in self.nodes:
            node.adjacent_nodes = set(node.adjacent_nodes)
        return
    
    # Algorithms
    # combine-then-adapt ASPM
    def caASPM(self, iters: int):
        return
        
    # adapt-then-combine LMS
    def acLMS(self, iters: int):
        for _ in range(iters):
            # sample
            for node in self.nodes:
                node.sample()
                node.y = np.dot(self.true_theta,node.x)+np.random.normal(0,node.noise)
            
            # compute error
            for node in self.nodes:
                for i in range(node.count_nodes):
                    node.error[i] = node.adjacent_nodes[i].y-np.dot(node.theta,node.adjacent_nodes[i].x)
            
            # compute psi
            for node in self.nodes:
                node.psi = np.zeros(60)
                for i in range(node.count_nodes):
                    node.psi += node.adjacent_vector[i]*node.error[i]*node.adjacent_nodes[i].x
                node.psi *= node.mu
                node.psi += node.theta
            
            # compute new theta
            for node in self.nodes:
                node.theta = np.zeros(60)
                for i in range(node.count_nodes):
                    node.theta += node.adjacent_vector[i]*node.adjacent_nodes[i].psi
            
            # compute msd and add into training history
            self.acLMS_history = np.append(self.acLMS_history,self.computeMSD())
            
        return
        
    # combine-then-adapt LMS
    def caLMS(self, iters: int):
        return
        
    # noncooperative LMS
    def ncLMS(self, iters: int):
        return
    
    # compute msd in dBs
    def computeMSD(self) -> float:
        msd = 0
        for node in self.nodes:
            msd += np.linalg.norm(self.true_theta-node.theta)**2
        msd = 10*np.log(msd/self.count_nodes)/np.log(10)
        return msd
    
    def networkInf(self):
        count_connections = (sum(node.count_nodes for node in self.nodes)-len(self.nodes))//2
        print("This network has %d nodes with %d connections."%(len(self.nodes),count_connections))
        print("The theta vector of this network is:\n",self.true_theta)
        return

class networkNode: add and delete adjacent nodes, generate sample, train the regressor $ \theta $, compute energy of the noise.

In [25]:
# networkNode
class networkNode:
    
    def __init__(self, num: int, controller: networkController):
        self.controller = controller
        self.num = num
        
        # main arguments
        self.theta = np.zeros(60) # initialise theta in R^60
        self.mu = 0.03 # initialise mu_k
        self.x = np.random.randn(60) # initialise sample vector
        self.y = 0
        self.error = np.zeros(0) # initialise error vector
        self.psi = np.zeros(60) # initialise psi vector
        self.snr = np.random.uniform(20,25) # initialise signal-to-noise ratio
        self.noise = 0 # initialise noise
        
        # adjacent nodes
        self.adjacent_nodes = set([self]) # initialise adjacent nodelist
        self.count_nodes = 1 # count adjacent nodes
        self.adjacent_vector = np.zeros(0) # initialise matrix C and A = C
    
    # connect self and nodes in node_list
    # elements in node_list are numbers of node
    def addAdjacentNode(self, node_list: list):
        for node_num in node_list:
            node = self.controller.nodes[node_num]
            self.adjacent_nodes.add(node)
            node.adjacent_nodes.add(self)
            node.count_nodes = len(node.adjacent_nodes)
        self.count_nodes = len(self.adjacent_nodes)
        return
    
    # disconnect self and nodes in node_list
    # elements in node_list are numbers of node
    def removeAdjacentNode(self, node_list: list):
        for node_num in node_list:
            node = self.controller.nodes[node_num]
            self.adjacent_nodes.discard(node)
            node.adjacent_nodes.discard(self)
            node.count_nodes = len(node.adjacent_nodes)
        self.adjacent_nodes.add(self)
        self.count_nodes = len(self.adjacent_nodes)
        return
    
    def sample(self):
        self.x = np.random.normal(0,1,60)
        return
    
    def updateArgs(self):
        # initialise a count_nodes-dimensionial error vector
        self.error = np.zeros(self.count_nodes)
        
        # select C and A = C
        self.adjacent_vector = np.zeros(self.count_nodes)
        for i in range(self.count_nodes):
            if self.adjacent_nodes[i].num != self.num:
                self.adjacent_vector[i] = 1/np.max([self.count_nodes,self.adjacent_nodes[i].count_nodes])
            else:
                mark = i
        self.adjacent_vector[mark] = 1-np.sum(self.adjacent_vector)
                
        return

In [26]:
controller = networkController()

In [27]:
controller.nodes[0].addAdjacentNode([1,4,6])
controller.nodes[1].addAdjacentNode([2,5,8])
controller.nodes[2].addAdjacentNode([3,7,9])
controller.nodes[3].addAdjacentNode([6,7,9])

In [28]:
controller.genTheta()
controller.networkInf()
controller.energy

This network has 10 nodes with 12 connections.
The theta vector of this network is:
 [-0.43304113 -0.25828102  0.18307782  1.58579912 -1.15826746 -0.05912153
  0.59890157 -0.99696607  1.41578565  0.7549235  -0.78043547 -0.06014652
  0.54492225  1.39773715  0.45870476  0.68222274 -0.11208242  0.54077115
 -1.64017579  0.07281858 -1.67465744 -0.27453032  0.15024011  0.50629895
 -0.01408815 -0.14667902 -1.46436473 -0.27040141  0.30543299 -0.69155901
 -0.79793405  0.02120931 -0.24765776  0.15938363  0.50136988 -0.96561754
 -0.44488243 -1.19028706 -1.5026153  -0.06135838 -0.59348351  1.4385069
  1.55203451 -1.35657012  0.79641561 -1.12839431  0.121248   -2.53931889
 -0.97958888  0.58698696 -0.52983594 -1.1174401   0.27498808  2.28428292
  1.10683548 -1.22469498 -0.27696699  0.75138975  0.79208712  0.23014173]


53.5824322387592

In [29]:
controller.startTrain(100,"acLMS")

In [30]:
controller.acLMS_history

array([ 0.00000000e+00,  1.71182739e+01,  1.68089488e+01,  1.67454765e+01,
        1.65440252e+01,  1.64448232e+01,  1.61467980e+01,  1.59088993e+01,
        1.57206094e+01,  1.55297235e+01,  1.53206552e+01,  1.51104712e+01,
        1.47250256e+01,  1.43695239e+01,  1.40826406e+01,  1.37714272e+01,
        1.34640338e+01,  1.30738718e+01,  1.27900959e+01,  1.24536181e+01,
        1.21820620e+01,  1.19435070e+01,  1.17264877e+01,  1.15509138e+01,
        1.13730622e+01,  1.12073221e+01,  1.09614048e+01,  1.05260922e+01,
        1.02067276e+01,  9.76112900e+00,  9.47112971e+00,  9.30073146e+00,
        8.93733019e+00,  8.67341791e+00,  8.52351898e+00,  8.33284087e+00,
        8.17461080e+00,  7.90439580e+00,  7.63073747e+00,  7.22622345e+00,
        7.07370137e+00,  6.80759189e+00,  6.53508601e+00,  6.23465431e+00,
        6.08885391e+00,  5.85345322e+00,  5.78692282e+00,  5.59041824e+00,
        5.23263911e+00,  5.02382812e+00,  4.75108722e+00,  4.08110025e+00,
        3.87567350e+00,  